In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from detector import MediaPipeProcessor
from config import Config
# ---------- Khởi tạo model ----------
model = MediaPipeProcessor(Config())
vid_not_kept = []
df_video = pd.DataFrame()

2025-06-30 19:53:41.676397: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 19:53:41.686890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751288021.701220   20211 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751288021.705094   20211 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-30 19:53:41.719020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
df = pd.read_csv('top50_daily.csv')

In [3]:
def moved(curr_kps: np.ndarray, ref_kps: np.ndarray, threshold=0.36) -> bool:
    """
    Trả về True nếu có ít nhất một phần tử của curr_kps khác ref_kps
    hơn ngưỡng threshold (normalized).
    """
    # So sánh độ chênh tuyệt đối từng chiều
    diffs = np.abs(curr_kps - ref_kps)
    return np.any(diffs > threshold)

In [4]:
def filter_video(id, src):
    cap = cv2.VideoCapture(src)
    # 1. Đọc khung đầu tiên, lấy keypoints làm reference

    ret, frame0 = cap.read()
    if not ret:
        print(f"✘ Không đọc được khung đầu của {src}")
        cap.release()
        return

    image0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2RGB)
    _, ref_res = model.process_frame(image0)
    ref_kps = model.extract_keypoints(ref_res)
    label = [0]
    kpts = [ref_kps]
    vid = []
    frame = 1
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb, res = model.process_frame(image_rgb)
        frame_processed = model.draw_landmarks(image_rgb, res)
        vid.append(frame_processed)
        if res.pose_landmarks:
            kpts_extract = model.extract_keypoints(res)
            if moved(kpts_extract, ref_kps):
                label.append(-1)
            else:
                label.append(0)
            kpts.append(kpts_extract)
    label = np.array(label)
    indices = np.where(label == -1)[0]
    if len(indices) > 0:
        last_idx = indices[-1]
        needed = 20

        # Số phần tử còn lại từ vị trí -1 đến hết
        after = len(label) - last_idx

        if after >= needed:
            label[last_idx : last_idx + needed] = id
        else:
            extra = needed - after
            start_idx = max(0, last_idx - extra)
            label[start_idx:] = id
    else:
        label[-20:] = id
    #all elemetnt = -1 turn to 0
    label[label == -1] = 0
    #all element in label += 1
    kpts = np.array(kpts)
    return kpts, label, vid

In [5]:
video_data = []
files_name = 'filtered_labels.csv'
df = pd.read_csv(files_name)
for i, row in df.iterrows():
    files_name = row['VIDEOS'].split('; ')
    id = row['STT']
    for file_name in files_name:
        file_name_output = file_name.split('.')[0]
        src = os.path.join('Dataset/Video', file_name)
        dst_vids = os.path.join('data/Videos', file_name_output + '.mp4')
        dst_kpts = os.path.join('data/Keypoints', file_name_output)
        dst_label = os.path.join('data/Labels', file_name_output)
        if not os.path.exists(src):
            print(f"✘ Không tìm thấy video: {src}")
            continue
        res_kpts, res_label, res_video = filter_video(id, src)
        res_kpts = res_kpts.reshape(res_kpts.shape[0], -1)
        #save keypoints and labels
        if res_kpts is None or res_label is None:
            print(f"✘ Không xử lý được video: {src}")
            continue
        #save video as mp4
        if not os.path.exists(os.path.dirname(dst_vids)):
            os.makedirs(os.path.dirname(dst_vids))
        writer = cv2.VideoWriter(dst_vids, cv2.VideoWriter_fourcc(*'mp4v'), 30, (res_video[0].shape[1], res_video[0].shape[0]))
        for fr in res_video:
            writer.write(cv2.cvtColor(fr, cv2.COLOR_RGB2BGR))
        writer.release()
        np.save(dst_kpts, res_kpts )
        np.save(dst_label, res_label)
        print(f'{id} - {file_name} - {res_kpts.shape} frames - {res_label.shape} labels')

1 - D0530.mp4 - (124, 150) frames - (124,) labels
2 - D0531.mp4 - (115, 150) frames - (115,) labels
3 - D0532.mp4 - (108, 150) frames - (108,) labels
4 - D0533.mp4 - (112, 150) frames - (112,) labels
5 - D0534.mp4 - (112, 150) frames - (112,) labels
6 - D0535N.mp4 - (101, 150) frames - (101,) labels
6 - D0535T.mp4 - (101, 150) frames - (101,) labels
6 - D0535B.mp4 - (150, 150) frames - (150,) labels
7 - D0536.mp4 - (105, 150) frames - (105,) labels
8 - D0537.mp4 - (121, 150) frames - (121,) labels
9 - D0538.mp4 - (113, 150) frames - (113,) labels
10 - D0489.mp4 - (93, 150) frames - (93,) labels
11 - D0492.mp4 - (93, 150) frames - (93,) labels
12 - W00325.mp4 - (97, 150) frames - (97,) labels
13 - D0493.mp4 - (82, 150) frames - (82,) labels
14 - W00570N.mp4 - (100, 150) frames - (100,) labels
14 - W00570T.mp4 - (119, 150) frames - (119,) labels
14 - W00570B.mp4 - (119, 150) frames - (119,) labels
15 - W00739N.mp4 - (71, 150) frames - (71,) labels
15 - W00739T.mp4 - (71, 150) frames - (7